In [ ]:
st.markdown("##")
st.markdown("<center><h5>DRAFT KINGS PROJECTIONS</h5></center>",unsafe_allow_html=True)

dg_proj_copy = pd.DataFrame(round(dg_proj_copy[['dk_name','dk_salary','total_points','value','projected_ownership']],2)).sort_values(by='dk_salary',ascending=False).reset_index(drop=True)
st.dataframe(dg_proj_copy.round(2).style.background_gradient(subset=['value'],cmap='Greys').format(precision=2),
             hide_index=True,
             height=1000,
             column_config={
                  'dk_name':'Name',
                  'dk_salary': st.column_config.NumberColumn(
                       'DK Salary',
                       format = "$ %.0f"
                    ),
                #    'early_late_wave':'Early/Late Wave',
                   'total_points':'Proj Pts',
                   'value': 'Value',
                   'projected_ownership':'pOwn',
                   'adj_from_baseline': 'Baseline Adj'
             },
             use_container_width=True
        )

In [20]:
import pandas as pd
import plotly.express as px
import numpy as np

In [21]:
dg_proj = pd.read_csv(f"https://feeds.datagolf.com/preds/fantasy-projection-defaults?tour=pga&site=draftkings&slate=main&file_format=csv&key=e297e933c3ad47d71ec1626c299e")
dg_proj = dg_proj[['player_name','proj_points_total']]
dg_proj

,player_name,proj_points_total
0,"McIlroy, Rory",72.77
1,"Matsuyama, Hideki",65.49
2,"Aberg, Ludvig",69.36
3,"Morikawa, Collin",65.09
4,"Homa, Max",62.81
...,...,...
151,"Teater, Josh",36.23
152,"Holmes, J.B.",28.94
153,"Gutschewski, Scott",32.77
154,"Sullivan, Joe",21.46


In [22]:
dg_proj.columns = ['player','proj_pts']
dg_proj

,player,proj_pts
0,"McIlroy, Rory",72.77
1,"Matsuyama, Hideki",65.49
2,"Aberg, Ludvig",69.36
3,"Morikawa, Collin",65.09
4,"Homa, Max",62.81
...,...,...
151,"Teater, Josh",36.23
152,"Holmes, J.B.",28.94
153,"Gutschewski, Scott",32.77
154,"Sullivan, Joe",21.46


In [23]:
def fix_names(dg):
        names = dg['player'].str.split(expand=True)                  
        names[0] = names[0].str.rstrip(",")
        names[1] = names[1].str.rstrip(",")
        names['player'] = names[1] + " " + names[0]

        names['player'] = np.where(names['player']=='Matt Fitzpatrick', 'Matthew Fitzpatrick', names['player'])
        names['player'] = np.where(names['player']=='Si Kim', 'Si Woo Kim', names['player'])
        names['player'] = np.where(names['player']=='Min Lee', 'Min Woo Lee', names['player'])
        names['player'] = np.where(names['player']=='Byeong An', 'Byeong Hun An', names['player'])
        names['player'] = np.where(names['player']=='Rooyen Van', 'Erik Van Rooyen', names['player'])
        names['player'] = np.where(names['player']=='Vince Whaley', 'Vincent Whaley', names['player'])
        names['player'] = np.where(names['player']=='Kevin Yu', 'kevin Yu', names['player'])
        names['player'] = np.where(names['player']=='Kyounghoon Lee', 'Kyoung-Hoon Lee', names['player'])
        return names.player

In [24]:
dg_proj.set_index(fix_names(dg_proj),inplace=True)

In [25]:
dg_proj

,player,proj_pts
player,,
Rory McIlroy,"McIlroy, Rory",72.77
Hideki Matsuyama,"Matsuyama, Hideki",65.49
Ludvig Aberg,"Aberg, Ludvig",69.36
Collin Morikawa,"Morikawa, Collin",65.09
Max Homa,"Homa, Max",62.81
...,...,...
Josh Teater,"Teater, Josh",36.23
J.B. Holmes,"Holmes, J.B.",28.94
Scott Gutschewski,"Gutschewski, Scott",32.77


In [26]:
usecols=['Player','Status','Roster Status']

teams = pd.read_csv(r"C:\Users\mikej\Desktop\fantrax\week.csv",usecols=usecols)
teams

,Player,Status,Roster Status
0,Ludvig Aberg,NT 4,Active
1,Sam Burns,[AW],Reserve
2,Patrick Cantlay,u_c,Active
3,Wyndham Clark,MG,Reserve
4,Eric Cole,MG,Active
...,...,...,...
115,Adrian Meronk,grrr,Reserve
116,Ben Silverman,grrr,Active
117,Gordon Sargent,NT 8,Reserve
118,Jacob Bridgeman,[AW],Reserve


In [27]:
teams_dict = {
        '919':'Philly919',
        'u_c':'unit_circle',
        'NT 4':'New Team 4',
        'NT 8':'Sneads Foot',
        'txms':'txmoonshine',
        'MG':'Team Gamble',
        'grrr':'Putt Pirates',
        '[AW]':'AlphaWired'
        }

In [28]:
teams.columns = ['player','team','active_reserve']
teams['team'] = teams.team.map(teams_dict)
teams.set_index('player',inplace=True)
teams

,team,active_reserve
player,,
Ludvig Aberg,New Team 4,Active
Sam Burns,AlphaWired,Reserve
Patrick Cantlay,unit_circle,Active
Wyndham Clark,Team Gamble,Reserve
Eric Cole,Team Gamble,Active
...,...,...
Adrian Meronk,Putt Pirates,Reserve
Ben Silverman,Putt Pirates,Active
Gordon Sargent,Sneads Foot,Reserve


In [29]:
####   MERGE DATA    ####
week = pd.merge(teams,dg_proj, left_index=True, right_index=True)

week[['player','team','active_reserve']] = week[
    ['player','team','active_reserve']
    ].astype('string')

week.sort_values('proj_pts',ascending=False,inplace=True)

In [30]:
week

,team,active_reserve,player,proj_pts
player,,,,
Rory McIlroy,Sneads Foot,Active,"McIlroy, Rory",72.77
Ludvig Aberg,New Team 4,Active,"Aberg, Ludvig",69.36
Jordan Spieth,AlphaWired,Active,"Spieth, Jordan",67.73
Alex Noren,unit_circle,Reserve,"Noren, Alex",66.34
Matthew Fitzpatrick,txmoonshine,Active,"Fitzpatrick, Matt",65.60
...,...,...,...,...
Cameron Champ,Sneads Foot,Reserve,"Champ, Cameron",44.88
Parker Coody,Sneads Foot,Reserve,"Coody, Parker",44.21
Brandon Wu,New Team 4,Active,"Wu, Brandon",43.32
